## Introduction to Quimb backend in QiboTN

#### Some imports

In [1]:
import time
import numpy as np
# from scipy import stats

# import qibo
from qibo import Circuit, gates, hamiltonians
from qibo.backends import construct_backend

#### Some hyper parameters

In [ ]:
import cotengra as ctg
ctg_opt = ctg.ReusableHyperOptimizer(
    max_time=10,
    minimize='combo',
    slicing_opts=None,
    parallel=True,
    progbar=True
)


In [2]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

# set number of qubits
nqubits = 4

# set numpy random seed
np.random.seed(42)

quimb_backend.setup_backend_specifics(
    qimb_backend="jax", 
    optimizer='auto-hq'
    )

#### Constructing a parametric quantum circuit

In [3]:
def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

In [4]:
circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [6]:
# Setting random parameters
circuit.set_parameters(
    parameters=np.random.uniform(-np.pi, np.pi, len(circuit.get_parameters())),
)

#### Setting up the tensor network simulator

Depending on the simulator, various parameters can be set. One can customize the tensor network execution via the `backend.configure_tn_simulation` function, whose face depends on the specific backend provider.

In [7]:
# Customization of the tensor network simulation in the case of quimb backend
# Here we use only some of the possible arguments
quimb_backend.configure_tn_simulation(
    #ansatz="MPS",
    max_bond_dimension=10
)

#### Executing through the backend

The `backend.execute_circuit` method can be used then. We can simulate results in three ways:
1. reconstruction of the final state only if `return_array` is set to `True`;
2. computation of the relevant probabilities of the final state.
3. reconstruction of the relevant state's frequencies (only if `nshots` is not `None`).

In [8]:
# # Simple execution (defaults)
outcome = quimb_backend.execute_circuit(circuit=circuit, nshots=100, return_array=True)

# # Print outcome
vars(outcome)

/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: creg
  warnings.warn(
/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: measure
  warnings.warn(
/home/mattia/main_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'nqubits': 4,
 'backend': qibotn (quimb),
 'measures': Counter({'0011': 8,
          '0010': 12,
          '0111': 4,
          '1011': 7,
          '0000': 8,
          '1110': 14,
          '0101': 4,
          '1010': 4,
          '1000': 14,
          '1111': 8,
          '0100': 6,
          '1101': 8,
          '1100': 1,
          '0110': 2}),
 'measured_probabilities': {'1110': np.float64(0.07174919872959985),
  '1000': np.float64(0.11330883548333587),
  '0010': np.float64(0.09466860481989385),
  '0011': np.float64(0.07571277233522114),
  '0000': np.float64(0.08390937969317269),
  '1111': np.float64(0.10184806171791962),
  '1101': np.float64(0.12331159869893256),
  '1011': np.float64(0.053499396925872744),
  '0100': np.float64(0.07142939529687138),
  '0111': np.float64(0.04029185074729259),
  '0101': np.float64(0.05622305772698622),
  '1010': np.float64(0.03872758515126756),
  '0110': np.float64(0.05146064807369214),
  '1100': np.float64(0.013605984872668404)},
 'prob_type': '

---

One can access to the specific contents of the simulation outcome.

In [9]:
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")

Probabilities:
 {'1110': np.float64(0.07174919872959985), '1000': np.float64(0.11330883548333587), '0010': np.float64(0.09466860481989385), '0011': np.float64(0.07571277233522114), '0000': np.float64(0.08390937969317269), '1111': np.float64(0.10184806171791962), '1101': np.float64(0.12331159869893256), '1011': np.float64(0.053499396925872744), '0100': np.float64(0.07142939529687138), '0111': np.float64(0.04029185074729259), '0101': np.float64(0.05622305772698622), '1010': np.float64(0.03872758515126756), '0110': np.float64(0.05146064807369214), '1100': np.float64(0.013605984872668404)}

State:
 [[ 0.08809624-0.27594998j]
 [-0.05174781+0.04471217j]
 [ 0.00470147+0.30764672j]
 [-0.27208942+0.0409893j ]
 [ 0.18807822+0.18988408j]
 [ 0.2237706 +0.07842042j]
 [-0.18900308+0.12545314j]
 [ 0.17105256-0.10503749j]
 [ 0.24859734-0.22695419j]
 [-0.0411739 -0.06230037j]
 [ 0.17371392-0.09247189j]
 [-0.22748128+0.0418529j ]
 [ 0.09444095+0.06846087j]
 [-0.21784972-0.2754144j ]
 [-0.17359753+0.2039

### Compute expectation values

Another important feature of this backend is the `expectation` function. In fact, we can compute expectation values of given observables thorugh a Qibo-friendly interface.

---

Let's start by importing some symbols, thanks to which we can build our observable.

In [10]:
# We are going to compute the expval of an Hamiltonian
# On the state prepared by the following circuit
circuit.draw()

circuit.set_parameters(
    np.random.randn(len(circuit.get_parameters()))
)

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [11]:
from qibo.symbols import Z, X, I
# We can create a symbolic Hamiltonian
form = 0.5 * Z(0) * Z(1) +- 1.5 *  X(0) * Z(2) + Z(3)
hamiltonian = hamiltonians.SymbolicHamiltonian(form)

#  Let's show it
hamiltonian.form

[Qibo 0.2.20|INFO|2025-09-20 16:43:42]: Using qibojit (numba) backend on /CPU:0


-1.5*X0*Z2 + 0.5*Z0*Z1 + Z3

In [13]:
start = time.time()
expval = quimb_backend.expectation(
    circuit=circuit,
    observable=hamiltonian,
)
elapsed = time.time() - start
print(f"Expectation value: {expval}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: 0.7143489122390747
Elapsed time: 12.4550 seconds


Try with Qibo (which is by default using the Qibojit backend)


In [14]:
start = time.time()
result = hamiltonian.expectation(circuit().state())
elapsed = time.time() - start
print(f"Expectation value: {result}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: 0.7143570920618565
Elapsed time: 0.5871 seconds


They match! 🥳

### Derivative of the extimation function

In [31]:
# grad of this circuit returning nan for some reason...

def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [ ]:
def build_circuit(nqubits, nlayers):
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
            circ.add(gates.RX(q=q, theta=0.))
        for q in range(nqubits - 1):
            circ.add(gates.CNOT(q, q + 1))
            circ.add(gates.SWAP(q, q + 1))
    circ.add(gates.M(*range(nqubits)))
    return circ

circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─RX─o─x─────────RY─RZ─RX─o─x─────────RY─RZ─RX─o─x─────────M─
1: ─RY─RZ─RX─X─x─o─x─────RY─RZ─RX─X─x─o─x─────RY─RZ─RX─X─x─o─x─────M─
2: ─RY─RZ─RX─────X─x─o─x─RY─RZ─RX─────X─x─o─x─RY─RZ─RX─────X─x─o─x─M─
3: ─RY─RZ─RX─────────X─x─RY─RZ─RX─────────X─x─RY─RZ─RX─────────X─x─M─


In [23]:
quimb_backend.expectation(
        circuit=circuit,         
        observable=hamiltonian,
   )

Array(1.4999985, dtype=float32)

In [29]:
import jax

def f(params):
    circuit.set_parameters(params)
    return quimb_backend.expectation(
        circuit=circuit,
        observable=hamiltonian,
    )

parameters = np.random.uniform(-np.pi, np.pi, size=len(circuit.get_parameters()))
print(jax.value_and_grad(f)(parameters))


(Array(0.4465402, dtype=float32), Array([-1.5755819e-01,  9.7801067e-02, -1.2350259e-01,  1.3670625e-01,
        3.6954228e-03, -1.7437905e-02,  2.7746204e-01, -1.0357879e-01,
        1.1504190e-01, -4.5175910e-02, -4.8447326e-02,  1.4743687e-01,
       -3.0708680e-01,  2.0652822e-01,  1.9298886e-01,  5.1306009e-02,
       -3.3362946e-01, -7.5548244e-01, -3.0034758e-02, -5.2868712e-01,
        4.8458660e-01, -2.9802322e-08,  8.0767423e-02,  0.0000000e+00],      dtype=float32))
